In [ ]:
import os
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
dataset = 'panx'

if dataset == 'pawsx':
    langs = ['english', 'german', 'chinese', 'french', 'japanese', 'korean', 'spanish']
elif dataset == 'xnli':
    langs = ['english', 'arabic', 'bulgarian', 'chinese', 'french', 'german', 'greek', 'hindi',
             'russian', 'spanish', 'swahili', 'thai', 'turkish', 'urdu', 'vietnamese']
elif dataset == 'udpos':
    langs = ['english', 'afrikaans', 'arabic', 'bulgarian', 'german', 'greek', 'spanish', 'estonian', 'basque', 'persian',
             'finnish', 'french', 'hebrew', 'hindi', 'hungarian', 'indonesian', 'italian', 'japanese', 'korean',
             'marathi', 'dutch', 'portuguese', 'russian', 'tamil', 'telugu', 'turkish', 'urdu', 'vietnamese', 'chinese']
elif dataset == 'panx':
    langs = ['english', 'afrikaans', 'arabic', 'bulgarian', 'bengali', 'german', 'greek', 'spanish', 'estonian', 'basque', 'persian',
             'finnish', 'french', 'hebrew', 'hindi', 'hungarian', 'indonesian', 'italian', 'japanese', 'javanese', 'georgian', 'kazakh', 'korean',
             'malayalam', 'marathi', 'malay', 'burmese', 'dutch', 'portuguese', 'russian', 'swahili', 'tamil', 'telugu',
             'thai', 'tagalog', 'turkish', 'urdu', 'vietnamese', 'yoruba', 'chinese']

In [ ]:
results_dict = {}
metrics = ['val_egs', 'tst_egs']
for lang in langs:
    results_dict[lang] = {}
    for metric in metrics:
        results_dict[lang][metric] = []
        
# update 1번 했을 때 성능 (0 epoch으로 둘려고..)
root_dir = '/input/jongwooko/xlt/checkpoint_baseline/{}/source_epoch'.format(dataset)
filelist = os.listdir(root_dir)
print (filelist)
file = [f for f in filelist if 'ftepcs-999' in f][0]
with open(os.path.join(root_dir, file, 'log.txt'), "r") as f:
    strings = f.readlines()
    evaluate_lst = [s for s in strings if 'Evaluate' in s]
    result_str = strings[strings.index(evaluate_lst[0])+2].split('dict_items')[-1].replace('\n', '')
result = ast.literal_eval(result_str)
for lang_idx, lang in enumerate(langs):
    for metric_idx, metric in enumerate(metrics):
        results_dict[lang][metric].append(result[lang_idx][1][metric_idx][1])

# update 1 epoch ~ 10 epoch 까지
file = [f for f in filelist if 'ftepcs-10' in f][0]
with open(os.path.join(root_dir, file, 'log.txt'), "r") as f:
    strings = f.readlines()
    evaluate_lst = [s for s in strings if 'Evaluate' in s]
    for e in evaluate_lst:
        result_str = strings[strings.index(e)+2].split('dict_items')[-1].replace('\n', '')
        result = ast.literal_eval(result_str)
        for lang_idx, lang in enumerate(langs):
            for metric_idx, metric in enumerate(metrics):
                results_dict[lang][metric].append(result[lang_idx][1][metric_idx][1])

In [ ]:
plt.figure(figsize=(12, 8))
metric_criterion = 'val_egs'
# metric_criterion = 'tst_egs'
for lang, metric in results_dict.items():
    xrange = range(len(metric[metric_criterion]))
    plt.plot(xrange, metric[metric_criterion], label=lang)
plt.legend()
plt.xlabel('Epoch')
if dataset in ['udpos', 'panx']:
    plt.ylabel('F1 score')
else:
    plt.ylabel('Accuracy')
plt.show()
plt.close()